In [1]:
# Import relevant packages
import torch
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import os
import time
import warnings
warnings.filterwarnings("ignore", category = FutureWarning)

In [2]:
# Flags
DISABLE_CUDA = False

In [3]:
# Hyperparameters
input_dim = 512
lr = 0.001
train_test_ratio = 0.8

# Declare important file paths
notebook_path = os.path.abspath("Custom_CNN.ipynb")
data_path = os.path.dirname(notebook_path) + '/data/columbia-prcg-datasets/'
model_path = os.path.dirname(notebook_path) + '/model.pth'

In [4]:
# Select accelerator device
def get_default_device():
    """Returns device, is_cuda (bool)."""
    if not DISABLE_CUDA and torch.cuda.is_available():
        print("Running on CUDA!")
        return torch.device('cuda'), True
    else:
        print("Running on CPU!")
        return torch.device('cpu'), False
device, using_cuda = get_default_device()

Running on CUDA!


In [5]:
print(device)

cuda


In [6]:
# Transform the data
transform = transforms.Compose([
                    transforms.Resize((input_dim, input_dim)),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Create training/testing dataloaders
full_set = datasets.ImageFolder(root=data_path, transform=transform)
train_size = int(train_test_ratio * len(full_set))
test_size = len(full_set) - train_size
train_set, test_set = torch.utils.data.random_split(full_set, [train_size, test_size])

train_loader = torch.utils.data.DataLoader(train_set, shuffle=True, num_workers=1)
test_loader = torch.utils.data.DataLoader(test_set, shuffle=False, num_workers=1)

In [7]:
# Declare our model architecture
import torch.nn as nn
import torch.nn.functional as F

class ConvNet(nn.Module):  # Convolutional Neural Network
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2),  # (512, 512, 32)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))  # (256, 256, 32)
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),  # (256, 256, 64)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))  #  (128, 128, 64)
        # self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(128 * 128 * 64, 1)
        # self.fc1 = nn.Linear(128 * 128 * 64, 1000)
        # self.fc2 = nn.Linear(1000, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        # out = self.drop_out(out)
        out = self.fc1(out)
        # out = self.fc2(out)
        out = self.sigmoid(out)
        return out

model = ConvNet()
model.to(device)

ConvNet(
  (layer1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=1048576, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [8]:
# Define the loss function and optimizer
loss_fn = torch.nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

In [9]:
loss_list = []
train_accuracy_list = []
test_accuracy_list = []
t = torch.Tensor([0.5]).to(device)  # 0.5 acts as threshold
highest_acc = 0.0

torch.backends.cudnn.benchmark = True  # make training faster on Cuda

start_time = time.time()

model.train()  # switch to train mode
    
for epoch in range(300):
    # Train the model
    running_loss = 0.0
    train_correct = train_total = 0 
    for i, data in enumerate(train_loader):
        inputs = data[0].to(device, non_blocking=True)
        labels = data[1].to(device, non_blocking=True)
    
        probs = model(inputs)

        outputs = (probs > t).float() * 1  # obtain train accuracies
        train_total += len(outputs)
        train_correct += (outputs == labels.float()).float().sum()

        loss = loss_fn(probs, labels.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if False and (i + 1) % 10 == 0:
            print ('# Images: {:} | Loss: {:.6f} | Time: {:.6f}'.format(i + 1, running_loss / (i + 1), time.time() - start_time))
    train_accuracy = train_correct / train_total
        
    # Test current version of model to obtain accuracy    
    test_correct = test_total = 0 
    with torch.no_grad():
        for data in test_loader:
            inputs = data[0].to(device, non_blocking=True)
            labels = data[1].to(device, non_blocking=True)
            probs = model(inputs)
            outputs = (probs > t).float() * 1
            test_total += len(outputs)
            test_correct += (outputs == labels.float()).float().sum()
    test_accuracy = test_correct / test_total

    if test_accuracy > highest_acc:  # save highest accuracy model
        highest_acc = test_accuracy
        torch.save(model.state_dict(), model_path)

    loss_list.append(running_loss)
    train_accuracy_list.append(train_accuracy)
    test_accuracy_list.append(test_accuracy)
    print ('Epoch: {:} | Time (m): {:.6f} | Loss: {:.6f} | Test Accuracy: {:}'.format(epoch, (time.time() - start_time)/60, running_loss, test_accuracy))

C:\Users\Gabe\Miniconda3\lib\site-packages\torch\nn\modules\loss.py:498: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


Epoch: 0 | Time (m): 0.860317 | Loss: 17368.711395 | Test Accuracy: 0.4625000059604645
Epoch: 1 | Time (m): 1.664934 | Loss: 17352.281502 | Test Accuracy: 0.4625000059604645
Epoch: 2 | Time (m): 2.462867 | Loss: 17352.281502 | Test Accuracy: 0.4625000059604645
Epoch: 3 | Time (m): 3.296167 | Loss: 17352.281502 | Test Accuracy: 0.4625000059604645
Epoch: 4 | Time (m): 4.107084 | Loss: 17352.281502 | Test Accuracy: 0.4625000059604645
Epoch: 5 | Time (m): 4.911251 | Loss: 17352.281502 | Test Accuracy: 0.4625000059604645
Epoch: 6 | Time (m): 5.726534 | Loss: 17352.281502 | Test Accuracy: 0.4625000059604645
Epoch: 7 | Time (m): 6.536151 | Loss: 17352.281502 | Test Accuracy: 0.4625000059604645
Epoch: 8 | Time (m): 7.389994 | Loss: 17352.281502 | Test Accuracy: 0.4625000059604645
Epoch: 9 | Time (m): 8.208157 | Loss: 17352.281502 | Test Accuracy: 0.4625000059604645
Epoch: 10 | Time (m): 9.025032 | Loss: 17352.281502 | Test Accuracy: 0.4625000059604645
Epoch: 11 | Time (m): 9.819536 | Loss: 173

Epoch: 92 | Time (m): 404.805272 | Loss: 17352.281502 | Test Accuracy: 0.4625000059604645
Epoch: 93 | Time (m): 405.656105 | Loss: 17352.281502 | Test Accuracy: 0.4625000059604645


KeyboardInterrupt: 

In [10]:
loss_list

[17368.711395263672,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,
 17352.28150177002,